## メモリリーク

- ヒープを利用するときに、用済みのデータ領域を開放せずにゴミが溜まる状態

## 二重解法

- すでに解放済みの領域を再度開放しようとすること

## ガベージコレクション

- ヒープの(?)のメモリ領域を開放すること
- 有効な領域から参照されていないメモリ領域を開放する

- https://ufcpp.net/study/computer/MemoryManagement.html
- Mark and Serrp
    - スコープ(有効な領域)から辿って、参照されていない領域を探してGCをする「マーク&スイープ」方式というのがある
    - rubyのGC
    - jsも
    -  https://developer.mozilla.org/ja/docs/Web/JavaScript/Memory_Management#Mark-and-sweep_algorithm
- Copy GC方式
    - ヒープ領域を2つとって、1箇所にまとめてコピーしていきながらコンパクションとGCを同時に行う
    - 片一方(ヒープ領域の半分)が未使用になってしまう
    - Java,OCaml,HaskellのGC
- 世代別GC
    - ヒープ領域を複数の世代に分ける
    - ruby,javaなど
- 参照カウント方式
    - https://developer.mozilla.org/ja/docs/Web/JavaScript/Memory_Management#Reference-counting_garbage_collection
    - 値を参照するたびにカウントを増やし、使わなくなったら減らし、0になったらfree
    - python,phpのGC
    - jsも
    
    
- https://developer.mozilla.org/ja/docs/Web/JavaScript/Memory_Management
- 難しいのは割り当てられたメモリが、今後必要とされない」ことを保証すること
- > メモリーの断片が不要になって解放する必要があるかを決定するには、開発者による判断が必要なことが多いです。


## JSの話

- 未読
- https://qiita.com/tkdn/items/ea4f034e0d661def244a


# ポインタ

- ポインタはアドレス変数
- 変数のアドレスを格納する変数

- (Cの場合)、必ず以下の3ステップで用いる
    1. 宣言
    2. 値(アドレス)の設定
    3. 使用
    
    
- `*`を使って型を指定してポインタを宣言
    - ポインタの差すものはただのアドレスなので、それの値の型がわからないので型を指定
```
var ptr *int
```

- `&`を使ってアドレスを取得
    - アンパサンドレス
```
pointer = &n // 変数nのアドレス
```

- `*`アドレスから変数の中身を見る
```
*pointer
    --------←あどれす
------------←値
```

- アドレス自体は演算できない(Goは)
    - `// pointer = pointer + 1               `    
    - `// tmp/sandbox390357081/main.go:37:20: invalid operation: pointer + 1 (mismatched types *int and int)  `
    
    
- 関数宣言時の引数に`*`がついているときはなに？？？？？
- func hoge(*a) {} ←宣言時。このとき*a, aはそれぞれ何？
- hoge(&a) ←呼び出し時。アドレスを渡している?
- https://fisproject.jp/2014/01/%E3%80%90c%E8%A8%80%E8%AA%9E%E3%80%91%E3%83%9D%E3%82%A4%E3%83%B3%E3%82%BF%E3%81%A8%E3%83%A1%E3%83%A2%E3%83%AA%E3%81%AE%E8%A9%B1%E3%80%90%E3%83%92%E3%83%BC%E3%83%97%E3%80%91/
    
## ポインタのポインタ

- `**hoge`
- ポインタ変数のアドレスを格納する変数
- ポインタのアドレスを書き換えたいときに使う
- https://bituse.info/c/28
    
    
## 値渡しと参照渡し

- 関数の引数に値をセットすると、関数内で値がコピーされる(値渡し)

- 関数の引数にポインタをセットすると、関数内での変更は元の値にも影響がある(参照渡し)
- https://www.yoheim.net/blog.php?q=20170901

# Rustの話

- http://vividcode.hatenablog.com/entry/rust/introduction/2
- https://doc.rust-jp.rs/the-rust-programming-language-ja/1.6/book/the-stack-and-the-heap.html
- http://south37.hatenablog.com/entry/2014/12/14/Rust%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8BBorrowing%E3%81%AE%E4%BB%95%E7%B5%84%E3%81%BF ←若干文法が古い
- RustにGCはない


- `&`をつけると参照になる
- 参照はいくつでも作れる
- https://qiita.com/cactaceae/items/2c70a9947364c60ec100
```
let v1 = vec![1, 2, 3];
let v2 = &v1;
let v3 = &v2;
println!("Hello, {}, {}, {} !", v1[0], v2[1], v3[2]); 
````

## ownership

- https://blog.waft.me/2017/09/24/rust-heap/
- C++ではコンパイルは通るが、実行すると落ちる
    - https://qiita.com/myhr47/items/262b8b8f1edd8097d03c
- http://mao-instantlife.hatenablog.com/entry/2016/05/05/Rust%E3%81%AE%E5%8B%89%E5%BC%B7%EF%BC%88%E3%81%9D%E3%81%AE%EF%BC%91%EF%BC%91%EF%BC%89
- Rustではムーブセマンティクスを通して1つの変数のみに所有権をもたせることで、リソースの二重解放を防いでいる

- 変数が開放されればデストラクタが呼ばれ、スタック、ヒープともに開放される
- 所有権があることによってデストラクトのタイミングを正確に把握できる
```
fn foo() {
    let v = vec![1, 2, 3];
}
```
- 新しいvectorはスタック上、各要素のためのスペースはヒープ上に確保
- スコープを抜ければ、vectorに関するリソースが解放される
    - スタック上のものは取り除かれ、ヒープ上のものものfreeされる

- https://qiita.com/kakaka19/items/2ba88ae0a148951d6f4c
    - http://ykomatsu.github.io/rust/book-ja/ownership.html

### そもそも束縛ってなに


- 以下の例の場合、「束縛されている値」は`vec![1,2,3]`のこと?
- `vec![1,2,3]`を名前`v`に束縛 ?
```
fn foo() {
    let v = vec![1, 2, 3];
}
```

## borrowing

- ownershipは移さずに値への参照を可能にする


## lifetime


## rustのポインタ

- 未読
- https://qiita.com/nebutalab/items/1d7a03c36c087c3f6360
- https://doc.rust-jp.rs/the-rust-programming-language-ja/1.6/book/raw-pointers.html
- https://qnighy.hatenablog.com/entry/2018/02/11/220000



# Goの話

## ポインタ

- https://qiita.com/Sekky0905/items/447efa04a95e3fec217f




- https://qiita.com/Sekky0905/items/c9cbda2498a685517ad0#%E3%83%9D%E3%82%A4%E3%83%B3%E3%82%BF
- `%p`でポインタ
```
fmt.Printf("ポインタ = %p", p)  // -> ポインタ = 0x1040a130

```

## mapとsliceの違い